<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Contents](Index.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# Aggregation and Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like ``sum()``, ``mean()``, ``median()``, ``min()``, and ``max()``, in which a single number gives insight into the nature of a potentially large dataset.
In this section, we'll explore aggregations in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a ``groupby``.

For convenience, we'll use the same ``display`` magic function that we've seen in previous sections:

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Data

Here we will use the Planets dataset, available via the [Seaborn package](http://seaborn.pydata.org/) (see [Visualization With Seaborn](04.14-Visualization-With-Seaborn.ipynb)).
It gives information on planets that astronomers have discovered around other stars (known as *extrasolar planets* or *exoplanets* for short). It can be downloaded with a simple Seaborn command:

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [40]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


This has some details on the 1,000+ extrasolar planets discovered up to 2014.

## Simple Aggregation in Pandas

Earlier, we explored some of the data aggregations available for NumPy arrays (["Aggregations: Min, Max, and Everything In Between"](02.04-Computation-on-arrays-aggregates.ipynb)).
As with a one-dimensional NumPy array, for a Pandas ``Series`` the aggregates return a single value:

In [107]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(100))
ser

0     0.374540
1     0.950714
2     0.731994
3     0.598658
4     0.156019
        ...   
95    0.493796
96    0.522733
97    0.427541
98    0.025419
99    0.107891
Length: 100, dtype: float64

In [108]:
ser.sum()

47.018074337820934

In [109]:
ser.mean()

0.4701807433782094

For a ``DataFrame``, by default the aggregates return results within each column:

In [110]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.031429,0.249292
1,0.636410,0.410383
2,0.314356,0.755551
3,0.508571,0.228798
4,0.907566,0.076980


In [111]:
df.mean()

A    0.479667
B    0.344201
dtype: float64

By specifying the ``axis`` argument, you can instead aggregate within each row:

In [46]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas ``Series`` and ``DataFrame``s include all of the common aggregates mentioned in [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb); in addition, there is a convenience method ``describe()`` that computes several common aggregates for each column and returns the result.
Let's use this on the Planets data, for now dropping rows with missing values:

In [112]:
planets.year.describe()

count    1035.000000
mean     2009.070531
std         3.972567
min      1989.000000
25%      2007.000000
50%      2010.000000
75%      2012.000000
max      2014.000000
Name: year, dtype: float64

In [47]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


This can be a useful way to begin understanding the overall properties of a dataset.
For example, we see in the ``year`` column that although exoplanets were discovered as far back as 1989, half of all known expolanets were not discovered until 2010 or after.
This is largely thanks to the *Kepler* mission, which is a space-based telescope specifically designed for finding eclipsing planets around other stars.

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

To go deeper into the data, however, simple aggregates are often not enough.
The next level of data summarization is the ``groupby`` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called ``groupby`` operation.
The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by Hadley Wickham of Rstats fame: *split, apply, combine*.

### Split, apply, combine

A canonical example of this split-apply-combine operation, where the "apply" is a summation aggregation, is illustrated in this figure:

![](figures/03.08-split-apply-combine.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

This makes clear what the ``groupby`` accomplishes:

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that *the intermediate splits do not need to be explicitly instantiated*. Rather, the ``GroupBy`` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way.
The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rather thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram.
We'll start by creating the input ``DataFrame``:

In [118]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


The most basic split-apply-combine operation can be computed with the ``groupby()`` method of ``DataFrame``s, passing the name of the desired key column:

In [119]:
df.groupby('key')

Notice that what is returned is not a set of ``DataFrame``s, but a ``DataFrameGroupBy`` object.
This object is where the magic is: you can think of it as a special view of the ``DataFrame``, which is poised to dig into the groups but does no actual computation until the aggregation is applied.
This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this ``DataFrameGroupBy`` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [121]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


The ``sum()`` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid ``DataFrame`` operation, as we will see in the following discussion.

### The GroupBy object

The ``GroupBy`` object is a very flexible abstraction.
In many ways, you can simply treat it as if it's a collection of ``DataFrame``s, and it does the difficult things under the hood. Let's see some examples using the Planets data.

Perhaps the most important operations made available by a ``GroupBy`` are *aggregate*, *filter*, *transform*, and *apply*.

#### Column indexing

The ``GroupBy`` object supports column indexing in the same way as the ``DataFrame``, and returns a modified ``GroupBy`` object.
For example:

In [124]:
gb_method = planets.groupby('method')
gb_method

In [125]:
planets.groupby('method')['orbital_period']

In [49]:
planets.groupby('method').orbital_period

Here we've selected a particular ``Series`` group from the original ``DataFrame`` group by reference to its column name.
As with the ``GroupBy`` object, no computation is done until we call some aggregate on the object:

In [127]:
planets.sample(10)

,method,number,orbital_period,mass,distance,year
897,Transit,1,3.005160,NaN,NaN,2013
523,Radial Velocity,1,1155.000000,0.36000,21.92,2005
850,Transit,4,13.570800,NaN,NaN,2014
488,Radial Velocity,1,1027.000000,0.85000,53.05,2009
145,Radial Velocity,4,124.260000,0.04600,4.70,2010
9,Radial Velocity,2,452.800000,1.99000,74.79,2010
2,Radial Velocity,1,763.000000,2.60000,19.84,2011
956,Transit,1,1.482245,NaN,360.00,2011
270,Radial Velocity,2,53.881000,0.06472,32.31,2011
185,Transit,1,3.646706,NaN,535.00,2011


In [132]:
planets.query('method == "Radial Velocity"').orbital_period.median()

360.2

In [126]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

This gives an idea of the general scale of orbital periods (in days) that each method is sensitive to.

#### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [7]:
planets.method.unique().tolist()

['Radial Velocity',
 'Imaging',
 'Eclipse Timing Variations',
 'Transit',
 'Astrometry',
 'Transit Timing Variations',
 'Orbital Brightness Modulation',
 'Microlensing',
 'Pulsar Timing',
 'Pulsation Timing Variations']

In [17]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


This can be useful for doing certain things manually, though it is often much faster to use the built-in ``apply`` functionality, which we will discuss momentarily.

#### Dispatch methods

Through some Python class magic, any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [137]:
planets.year.describe()

count    1035.000000
mean     2009.070531
std         3.972567
min      1989.000000
25%      2007.000000
50%      2010.000000
75%      2012.000000
max      2014.000000
Name: year, dtype: float64

In [11]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


Looking at this table helps us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and Transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade.
The newest methods seem to be Transit Timing Variation and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.

This is just one example of the utility of dispatch methods.
Notice that they are applied *to each individual group*, and the results are then combined within ``GroupBy`` and returned.
Again, any valid ``DataFrame``/``Series`` method can be used on the corresponding ``GroupBy`` object, which allows for some very flexible and powerful operations!

### Aggregate, filter, transform, apply

The preceding discussion focused on aggregation for the combine operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this ``DataFrame``:

In [196]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C','A'],
                   'data1': range(7),
                   'data2': rng.randint(0, 10, 7)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
6,A,6,3


#### Aggregation

We're now familiar with ``GroupBy`` aggregations with ``sum()``, ``median()``, and the like, but the ``aggregate()`` method allows for even more flexibility.
It can take a string, a function, or a list thereof, and compute all the aggregates at once.
Here is a quick example combining all these:

In [143]:
a = df.groupby('key').data1.min().rename('min')
b = df.groupby('key').data1.max().rename('max')
c = df.groupby('key').data1.median().rename('median')
pd.concat([a,b,c], axis=1)

,min,max,median
key,,,
A,0,3,1.5
B,1,4,2.5
C,2,5,3.5


In [155]:
df.groupby('key').data1.agg(['min', np.median, max])

,min,median,max
key,,,
A,0,1.5,3
B,1,2.5,4
C,2,3.5,5


In [153]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [193]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [157]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': ['min', 'max']})

data1 data2    
      min   min max
key                
A       0     3   5
B       1     0   7
C       2     3   9

#### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [161]:
filter_func(df.query('key == "C"'))

True

In [164]:
def filter_func(grp, p=4):
    return grp['data2'].std() > p

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

The filter function should return a Boolean value specifying whether the group passes the filtering. Here because group A does not have a standard deviation greater than 4, it is dropped from the result.

#### Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.
For such a transformation, the output is the same shape as the input.
A common example is to center the data by subtracting the group-wise mean:

In [176]:
pd.concat([df['key'], df.groupby('key').transform(lambda x: x - x.mean())], axis=1)

,key,data1,data2
0,A,-1.5,1.0
1,B,-1.5,-3.5
2,C,-1.5,-3.0
3,A,1.5,-1.0
4,B,1.5,3.5
5,C,1.5,3.0


#### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

For example, here is an ``apply()`` that normalizes the first column by the sum of the second:

In [198]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] = x['data1'] /  x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
6,A,6,3
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0


### Applying any function

We can apply any function, even our own function. The only thing we need to make sure is that if our function retruns more than one value, we return them as a Pandas object: 

Do not do this:

In [200]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [202]:
def get_sum_and_mean(grp):
    return grp.orbital_period.sum(), grp.number.mean()    

In [203]:
not_good = planets.groupby('method').apply(get_sum_and_mean).head()
not_good

method
Astrometry                              (1262.3600000000001, 1.0)
Eclipse Timing Variations           (42764.8, 1.6666666666666667)
Imaging                          (1418972.85, 1.3157894736842106)
Microlensing                         (22075.0, 1.173913043478261)
Orbital Brightness Modulation    (2.12791975, 1.6666666666666667)
dtype: object

Because the returned object is not further analyazble. It is a series, but with tuple walues:

Instead, return a `pd.Series` object. Pandas will tranlate it to columns automatically:

In [205]:
pd.Series({'sum': 5, 'avg': 6})

sum    5
avg    6
dtype: int64

In [206]:
def get_sum_and_mean(grp):
    return pd.Series({'sum': grp.orbital_period.sum(), 'avg': grp.number.mean()})

In [209]:
good = planets.groupby('method').apply(get_sum_and_mean).head()
good

,sum,avg
method,,
Astrometry,1.262360e+03,1.000000
Eclipse Timing Variations,4.276480e+04,1.666667
Imaging,1.418973e+06,1.315789
Microlensing,2.207500e+04,1.173913
Orbital Brightness Modulation,2.127920e+00,1.666667


``apply()`` within a ``GroupBy`` is quite flexible: the only criterion is that the function takes a ``DataFrame`` and returns a Pandas object or scalar; what you do in the middle is up to you!

### Specifying the split key

In the simple examples presented before, we split the ``DataFrame`` on a single column name.
This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

#### A list, array, series, or index providing the grouping keys

The key can be any series or list with a length matching that of the ``DataFrame``. For example:

In [29]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

But purheps a more explicit way is better:

In [30]:
df1 = df.copy()
df1['groupby_key'] = L
display('df1', 'df1.groupby("groupby_key").sum()')

df1
  key  data1  data2  groupby_key
0   A      0      5            0
1   B      1      0            1
2   C      2      3            0
3   A      3      3            1
4   B      4      7            2
5   C      5      9            0

df1.groupby("groupby_key").sum()
             data1  data2
groupby_key              
0                7     17
1                4      3
2                4      7

Of course, this means there's another, more verbose way of accomplishing the ``df.groupby('key')`` from before:

#### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys:

In [31]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
key,,
consonant,12,19


Again, a similar result can be achieved by:

In [35]:
df2['gb_key'] = df2.index.map(mapping)
display('df2', 'df2.groupby("gb_key").sum()')

df2
     data1  data2     gb_key
key                         
A        0      5      vowel
B        1      0  consonant
C        2      3  consonant
A        3      3      vowel
B        4      7  consonant
C        5      9  consonant

df2.groupby("gb_key").sum()
           data1  data2
gb_key                 
consonant     12     19
vowel          3      8

#### Any Python function

Similar to mapping, you can pass any Python function that will input the index value and output the group:

In [28]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


### Grouping example

As an example of this, in a couple lines of Python code we can put all these together and count discovered planets by method and by decade:

In [210]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'

In [211]:
planets['decade'] = decade
planets.head()

,method,number,orbital_period,mass,distance,year,decade
0,Radial Velocity,1,269.300,7.10,77.40,2006,2000s
1,Radial Velocity,1,874.774,2.21,56.95,2008,2000s
2,Radial Velocity,1,763.000,2.60,19.84,2011,2010s
3,Radial Velocity,1,326.030,19.40,110.62,2007,2000s
4,Radial Velocity,1,516.220,10.50,119.47,2009,2000s


In [212]:
planets.groupby(['method', 'decade'])['number'].sum()#.unstack().fillna(0)

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

This shows the power of combining many of the operations we've discussed up to this point when looking at realistic datasets.
We immediately gain a coarse understanding of when and how planets have been discovered over the past several decades!

Here I would suggest digging into these few lines of code, and evaluating the individual steps to make sure you understand exactly what they are doing to the result.
It's certainly a somewhat complicated example, but understanding these pieces will give you the means to similarly explore your own data.

## Groupby and Multi-Index

a groupy combine step will output a dataframe in which the requested groupby key, or keys, are at the index. This can be quite convinient for further querying:

In [217]:
planets.groupby(['method', 'decade'])['number'].sum()

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

In [214]:
planets.groupby(['method', decade])['number'].sum().loc['Radial Velocity']

year
1980s      1
1990s     52
2000s    475
2010s    424
Name: number, dtype: int64

This is convinient, but in many cases we would better work with a dataframe and not a Series. We can convert our series to a dataframe using the `.to_frame()` method. Note - The name of the series will become the column name:

In [218]:
planets.groupby(['method', decade])['number'].sum().to_frame().head()

number
method                    year         
Astrometry                2010s       2
Eclipse Timing Variations 2000s       5
                          2010s      10
Imaging                   2000s      29
                          2010s      21

To rename, use the `.rename()` method:

In [89]:
planets.groupby(['method', decade])['number'].sum().rename('total').to_frame().head()

total
method                    decade       
Astrometry                2010s       2
Eclipse Timing Variations 2000s       5
                          2010s      10
Imaging                   2000s      29
                          2010s      21

If we will aggregate more than one function, we will get a dataframe and not series as the output:

In [90]:
sum_and_max_per_method = planets.groupby(['method', decade])['number'].agg(['sum', 'max'])
sum_and_max_per_method.head()

sum  max
method                    decade          
Astrometry                2010s     2    1
Eclipse Timing Variations 2000s     5    2
                          2010s    10    2
Imaging                   2000s    29    4
                          2010s    21    4

In [224]:
sum_and_max_per_method.loc['Microlensing']

,sum,max
decade,,
2000s,12,2
2010s,15,2


sum_and_max_per_method is a dataframe with a multi-index:

In [93]:
sum_and_max_per_method.index[:5]

MultiIndex([(               'Astrometry', '2010s'),
            ('Eclipse Timing Variations', '2000s'),
            ('Eclipse Timing Variations', '2010s'),
            (                  'Imaging', '2000s'),
            (                  'Imaging', '2010s')],
           names=['method', 'decade'])

We can transfer the index to the columns using the `reset_index()` method:

In [95]:
sum_and_max_per_method.reset_index().head()

,method,decade,sum,max
0,Astrometry,2010s,2,1
1,Eclipse Timing Variations,2000s,5,2
2,Eclipse Timing Variations,2010s,10,2
3,Imaging,2000s,29,4
4,Imaging,2010s,21,4


If we want to move only part of the multi-index to the columns, we can specificy it to the function call:

In [98]:
sum_and_max_per_method.reset_index('method').head()

,method,sum,max
decade,,,
2010s,Astrometry,2,1
2000s,Eclipse Timing Variations,5,2
2010s,Eclipse Timing Variations,10,2
2000s,Imaging,29,4
2010s,Imaging,21,4


In [99]:
sum_and_max_per_method.reset_index('decade').head()

,decade,sum,max
method,,,
Astrometry,2010s,2,1
Eclipse Timing Variations,2000s,5,2
Eclipse Timing Variations,2010s,10,2
Imaging,2000s,29,4
Imaging,2010s,21,4


Unstacking moves the most inner index to the columns, and create a multi-column dataframe:

In [227]:
sum_and_max_per_method

sum  max
method                        decade          
Astrometry                    2010s     2    1
Eclipse Timing Variations     2000s     5    2
                              2010s    10    2
Imaging                       2000s    29    4
                              2010s    21    4
Microlensing                  2000s    12    2
                              2010s    15    2
Orbital Brightness Modulation 2010s     5    2
Pulsar Timing                 1990s     9    3
                              2000s     1    1
                              2010s     1    1
Pulsation Timing Variations   2000s     1    1
Radial Velocity               1980s     1    1
                              1990s    52    5
                              2000s   475    6
                              2010s   424    6
Transit                       2000s    64    2
                              2010s   712    7
Transit Timing Variations     2010s     9    3

In [229]:
sum_and_max_per_method.unstack().head()

sum                     max                  
decade                        1980s 1990s 2000s 2010s 1980s 1990s 2000s 2010s
method                                                                       
Astrometry                      NaN   NaN   NaN   2.0   NaN   NaN   NaN   1.0
Eclipse Timing Variations       NaN   NaN   5.0  10.0   NaN   NaN   2.0   2.0
Imaging                         NaN   NaN  29.0  21.0   NaN   NaN   4.0   4.0
Microlensing                    NaN   NaN  12.0  15.0   NaN   NaN   2.0   2.0
Orbital Brightness Modulation   NaN   NaN   NaN   5.0   NaN   NaN   NaN   2.0

In [103]:
sum_and_max_per_method.unstack('method').head()

sum                                                 \
method Astrometry Eclipse Timing Variations Imaging Microlensing   
decade                                                             
1980s         NaN                       NaN     NaN          NaN   
1990s         NaN                       NaN     NaN          NaN   
2000s         NaN                       5.0    29.0         12.0   
2010s         2.0                      10.0    21.0         15.0   

                                                    \
method Orbital Brightness Modulation Pulsar Timing   
decade                                               
1980s                            NaN           NaN   
1990s                            NaN           9.0   
2000s                            NaN           1.0   
2010s                            5.0           1.0   

                                                            \
method Pulsation Timing Variations Radial Velocity Transit   
decade                                                       
1980s                          NaN             1.0     NaN   
1990s                          NaN            52.0     NaN   
2000s                          1.0           475.0    64.0   
2010s                          NaN           424.0   712.0   

                                        max                                    \
method Transit Timing Variations Astrometry Eclipse Timing Variations Imaging   
decade                                                                          
1980s                        NaN        NaN                       NaN     NaN   
1990s                        NaN        NaN                       NaN     NaN   
2000s                        NaN        NaN                       2.0     4.0   
2010s                        9.0        1.0                       2.0     4.0   

                                                                 \
method Microlensing Orbital Brightness Modulation Pulsar Timing   
decade                                                            
1980s           NaN                           NaN           NaN   
1990s           NaN                           NaN           3.0   
2000s           2.0                           NaN           1.0   
2010s           2.0                           2.0           1.0   

                                                            \
method Pulsation Timing Variations Radial Velocity Transit   
decade                                                       
1980s                          NaN             1.0     NaN   
1990s                          NaN             5.0     NaN   
2000s                          1.0             6.0     2.0   
2010s                          NaN             6.0     7.0   

                                  
method Transit Timing Variations  
decade                            
1980s                        NaN  
1990s                        NaN  
2000s                        NaN  
2010s                        3.0

In [233]:
sum_and_max_per_method.unstack('decade').stack()

sum  max
method                        decade            
Astrometry                    2010s     2.0  1.0
Eclipse Timing Variations     2000s     5.0  2.0
                              2010s    10.0  2.0
Imaging                       2000s    29.0  4.0
                              2010s    21.0  4.0
Microlensing                  2000s    12.0  2.0
                              2010s    15.0  2.0
Orbital Brightness Modulation 2010s     5.0  2.0
Pulsar Timing                 1990s     9.0  3.0
                              2000s     1.0  1.0
                              2010s     1.0  1.0
Pulsation Timing Variations   2000s     1.0  1.0
Radial Velocity               1980s     1.0  1.0
                              1990s    52.0  5.0
                              2000s   475.0  6.0
                              2010s   424.0  6.0
Transit                       2000s    64.0  2.0
                              2010s   712.0  7.0
Transit Timing Variations     2010s     9.0  3.0

In [73]:
planets.groupby(['method', decade])['number'].sum().rename('total_number').unstack('decade').stack().head()

method                     decade
Astrometry                 2010s      2.0
Eclipse Timing Variations  2000s      5.0
                           2010s     10.0
Imaging                    2000s     29.0
                           2010s     21.0
dtype: float64

<!--NAVIGATION-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Contents](Index.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
